In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Imports

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import ensemble
from sklearn import metrics
from sklearn import model_selection
from sklearn import decomposition
from sklearn import preprocessing
from sklearn import base
from sklearn import svm

from sklearn import pipeline
from functools import partial
from skopt import space
from skopt import gp_minimize
from skopt import BayesSearchCV

import optuna

Feature engineering

In [ ]:
df = pd.read_csv('/kaggle/input/water-potability/water_potability.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.nunique()

In [ ]:
df.isnull().sum()

In [ ]:
def percent_missing(df):
    
    precent_nan = 100*df.isnull().sum() / len(df)
    precent_nan = precent_nan[precent_nan>0].sort_values()
    
    return precent_nan

In [ ]:
percent_missing(df)и

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [ ]:
imputer = IterativeImputer(estimator=ensemble.RandomForestRegressor(), imputation_order='ascending')

In [ ]:
X = df.drop('Potability',axis=1)
y = df['Potability']

In [ ]:
imputer.fit(X)

In [ ]:
Xtrans = imputer.transform(X)

In [ ]:
dframe = pd.DataFrame(Xtrans,columns=df.columns[0:-1])

In [ ]:
dframe['Potability'] = y

In [ ]:
percent_missing(dframe)

In [ ]:
sns.pairplot(dframe,hue='Potability')

Standart RF

In [ ]:
X = dframe.drop('Potability',axis=1)
y = dframe['Potability']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
scaler = preprocessing.StandardScaler()

In [ ]:
scaled_X_train = scaler.fit_transform(X_train)
scaled_X_test = scaler.transform(X_test)

In [ ]:
classifier = ensemble.RandomForestClassifier(n_jobs=-1)

In [ ]:
classifier.fit(scaled_X_train,y_train)

In [ ]:
y_preds = classifier.predict(scaled_X_test)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,y_preds))
confusion_matrix(y_test,y_preds)

In [ ]:
plt.figure(figsize=(12,8),dpi=150)
feat_importances = pd.Series(classifier.feature_importances_, index=X.columns)
feat_importances.nlargest(30).plot(kind='barh')

RF + Bayesian Optimization with Gaussian Process

In [ ]:
def optimize(params,param_names,x,y):
    params = dict(zip(param_names,params))
    model = ensemble.RandomForestClassifier(**params)
    kf = model_selection.StratifiedKFold(n_splits=5)
    accuracies = []
    for idx in kf.split(X=x,y=y):
        train_idx,test_idx = idx[0],idx[1]
        
        xtrain = x.iloc[train_idx]
        ytrain = y.iloc[train_idx]
        
        xtest=x.iloc[test_idx]
        ytest=y.iloc[test_idx]
        
        model.fit(xtrain,ytrain)
        preds = model.predict(xtest)
        fold_acc = metrics.accuracy_score(ytest,preds)
        accuracies.append(fold_acc)
    return -1.0 * np.mean(accuracies)

In [ ]:
param_space = [
    space.Integer(2,30,name='max_depth'),
    space.Integer(100,1500,name='n_estimators'),
    space.Integer(1,20,name='min_samples_leaf'),
    space.Categorical(['gini','entropy'],name='criterion'),
    space.Real(0.01,1,prior='uniform',name='max_features')
    
]

In [ ]:
param_names= [
    'max_depth',
    'n_estimators',
    'min_samples_leaf',
    'criterion',
    'max_features'
]

In [ ]:
optimization_function = partial(
optimize,
param_names=param_names,
x=X,
y=y)

In [ ]:
optimization_function = partial(optimize,param_names=param_names,x=X,y=y)

In [ ]:
result = gp_minimize(
optimization_function,
dimensions=param_space,
n_calls=30,
n_random_starts=10,
verbose=0)

print(
dict(zip(param_names,result.x))
)

RF + Optuna

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
def rf_objective(trial):
    params = {
        "n_estimators": trial.suggest_int('n_estimators', 10, 100),
        "max_depth": trial.suggest_categorical("max_depth", [7, 8, 9, 10, 11, 12, None]),
        "criterion": trial.suggest_categorical('criterion', ["gini", "entropy"]),
        "min_samples_split": trial.suggest_int('min_samples_split', 2, 5),
        "min_samples_leaf": trial.suggest_categorical('min_samples_leaf', [1, 2]),
        "max_features": trial.suggest_categorical('max_features', ["auto", "sqrt", "log2"]),
        "class_weight": trial.suggest_categorical('class_weight', ["balanced"]),
        "random_state": trial.suggest_categorical('random_state', [0]),
        "n_jobs": trial.suggest_categorical('n_jobs', [-1]),
    }
    model = ensemble.RandomForestClassifier(**params)
    model.fit(X_train, y_train)
    return -accuracy_score(y_test, model.predict(X_test))

In [ ]:
study = optuna.create_study()

In [ ]:
study.optimize(rf_objective, n_trials=200, timeout=3600 * 2)
print(f"Best RandomForest accuracy: {-round(study.best_value, 4)} with parameters {study.best_params}\n\n")

SVC

In [ ]:
svc = svm.SVC()

In [ ]:
svc.fit(scaled_X_train,y_train)

In [ ]:
svc_preds = svc.predict(scaled_X_test)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,svc_preds))
confusion_matrix(y_test,svc_preds)

SVC + Optuna

In [ ]:
def svc_objective(trial):
    params = {
        "C": trial.suggest_float('C', 1e-10, 1e10, log=True),
        "kernel": trial.suggest_categorical('kernel',['linear', 'poly', 'rbf', 'sigmoid']),
        "degree": trial.suggest_int('degree',1,20,step=1 ),
        "gamma": trial.suggest_categorical("gamma",['scale', 'auto']),
        "max_iter": trial.suggest_int('max_iter',1,10000,step=10),
        "decision_function_shape": trial.suggest_categorical('decision_function_shape', ['ovo', 'ovr']),
        "class_weight": trial.suggest_categorical('class_weight', ["balanced"]),
        "random_state": trial.suggest_categorical('random_state', [0]),
    }
    model = svm.SVC(**params)
    model.fit(scaled_X_train, y_train)
    return -accuracy_score(y_test, model.predict(scaled_X_test))

In [ ]:
other_study = optuna.create_study()

In [ ]:
other_study.optimize(svc_objective, n_trials=500, timeout=3600 * 2)
print(f"Best svc accuracy: {-round(other_study.best_value, 4)} with parameters {other_study.best_params}\n\n")